In [1]:
import numpy as np
import sympy as sp
from numba import njit
from scipy import linalg

# Opreações Matriciais

In [12]:
mat33 = np.array([[1,2,3], [5,6,3], [4,8,1]])
mat43 = np.array([[1,2,3], [5,6,3], [4,8,1], [4,-1,10]])
arr = np.array([3,4,5])

In [13]:
mat33

array([[1, 2, 3],
       [5, 6, 3],
       [4, 8, 1]])

## Traço de Matrix

In [14]:
def matrix_trace(a):
    assert a.shape[0] == a.shape[1], 'Matrix must be quadratic'
    sum = 0
    for i in range(len(a)):
        sum += a[i][i]
    
    return sum

In [15]:
matrix_trace(mat33)

8

In [16]:
mat33.trace()

8

## Transposto de Matrix

In [17]:
def matrix_tranpose(a):
    res = np.empty((a.shape[1], a.shape[0]))
    for idx, row in enumerate(a):
        for i, element in enumerate(row):
            res[i, idx] = element

    return res

In [18]:
matrix_tranpose(mat43)

array([[ 1.,  5.,  4.,  4.],
       [ 2.,  6.,  8., -1.],
       [ 3.,  3.,  1., 10.]])

In [19]:
mat43.T

array([[ 1,  5,  4,  4],
       [ 2,  6,  8, -1],
       [ 3,  3,  1, 10]])

## Produto Matricial

In [51]:
@njit()
def matrix_prod(a, b):
    assert a.shape[1] == b.shape[0], "The product is not defined"
    res = np.empty((a.shape[0], b.shape[1]))
    for idx_r, row in enumerate(a):
        for idx_c, col in enumerate(b.T):
            sum = 0
            for i, j in zip(row, col):
                sum += i*j
            res[idx_r, idx_c] = sum
    
    return res


In [52]:
%timeit matrix_prod(mat33, mat33)

631 ns ± 21 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [53]:
%timeit mat33.dot(mat33)

555 ns ± 24.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


# Representando Equações Algébricas Lineares na Forma Matricial

Um sistema de equações lineares pode ser representado como um produto matricual. Vamos ver atraves de um exemplo. 

O sistema abaixo

$
\begin{align*}
2x + 3y - z = 2 \\
x - 4y + 2z = 5 \\
-3x - y + 9z = -3
\end{align*}
$

Pode ser escrito como

$
A = \begin{bmatrix}
2 & 3 & -1 \\
1 & -4 & 2 \\
-3 & -1 & 9 
\end{bmatrix}  
$

$
X = \begin{bmatrix}
x  \\
y  \\
z  
\end{bmatrix}
$

$
B = \begin{bmatrix}
2  \\
5  \\
-3  
\end{bmatrix}
$

Aí, com essas definições podem reescrever o sistema na forma matricial

$\Rightarrow [A]{x} = {b}$

Agora, se multiplicarmos os dois lados por $A^{-1}$ teremos

$A^{-1} A X = A^{-1} B \rightarrow X = A^{-1} B$

Então, a equação foi resolvida e $X$ for determinado. Esse é um outro exemplo de como
a inversa desempenha um papel na álgebra de matrizes que é semelhante à divisão. Devemos observar que esse não é um método muito eficiente para resolver um sistema de equações. Portanto, outras abordagens são empregadas nos algoritmos numéricos.

# Eliminação de Gauss

Este metodo envolve combinar equações para eliminar variáveis. Apesar de ser um dos métodos mais antigos para resolver equações simultâneas, mantém-se como um dos algoritmos mais importantes em uso hoje em dia e é a base da resolução de equações lineares em muitos pacotes de software populares.


## Determinantes e a Regra de Cramer

Determinante é um conceito muito importante em calculo matricial. Vamos ver como esse conceito nos ajuda a descobrir se um sistema tem solução ou não. E alem disso, precisaremos dele para implementar a regra de Cramer para resolver um sistema de eqs. lineares.

O determinante de uma matrix 2x2 é definido como

$D = \begin{vmatrix}
a_{11} & a_{12} \\
a_{21} & a_{22}  
\end{vmatrix} 
 = a_{11}a_{22} - a_{12}a_{21}$

E para uma matrix 3x3 é 

$$
D = \begin{vmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33} 
\end{vmatrix} 
= 
a_{11}\begin{vmatrix}
a_{22} & a_{23} \\
a_{32} & a_{33}  
\end{vmatrix}  - 
a_{12}\begin{vmatrix}
a_{21} & a_{23} \\
a_{31} & a_{33}  
\end{vmatrix} + 
a_{13}\begin{vmatrix}
a_{21} & a_{22} \\
a_{31} & a_{32}  
\end{vmatrix}
$$

In [23]:
def matrix_determinent(a):
    assert a.shape[0] == a.shape[1], 'Matrix must be quadratic'
    assert len(a) == 2 or len(a) == 3, 'The shape of matrix must be 2x2 or 3x3'

    if len(a) == 2:
        return a[0,0]*a[1,1] - a[0,1]*a[1,0]
    if len(a) == 3:
        return a[0,0]*matrix_determinent(a[1:,[1,2]]) - a[0,1]*matrix_determinent(a[1:,[0,2]])+\
            a[0,2]*matrix_determinent(a[1:,[0,1]])


In [24]:
matrix_determinent(mat33)

44

In [25]:
np.linalg.det(mat33)

43.99999999999997

Um sistema se diz **singular** se tiver determinante nulo. Neste caso, o sistema não tem uma solução unica. 

## Regra de Cramer. 
Essa regra estabelece que cada incógnita em um sistema de equações algébricas lineares pode ser expressa como uma fração de dois determinantes, com denominador $D$ e com o numerador obtido a partir de $D$ trocando-se a coluna de coeficientes da incógnita em questão pelas constantes $b_1, b_2,..., b_n$ . Por exemplo, $x_1$ é calculada por
$$
x_1 = \frac{\begin{vmatrix}
b_{1} & a_{12} & a_{13}\\
b_{2} & a_{22} & a_{23} \\
b_{3} & a_{32} & a_{33} 
\end{vmatrix} }{D}
$$

### **Exemplo:** Use a regra de Cramer para resolver

$$
\begin{align}
0.3 x_1 &+ 0.52 x_2 + x_3 = -0.01 \\
0.5 x_1 &+ x_2 + 1.9 x_3 = 0.67 \\
0.1 x_1 &+ 0.3 x_2 + 0.5 x_3 = -0.44
\end{align}
$$



In [26]:
def cramer(a, b):
    assert a.shape[0] == a.shape[1], 'Matrix must be quadratic'
    assert a.shape[0] == b.shape[0], 'The dimension of the matrix must the same as the vector of constants'
    D = matrix_determinent(a)
    temp = a.copy()
    temp[:,0] = b
    x1 = matrix_determinent(temp)/D

    temp = a.copy()
    temp[:,1] = b
    x2 = matrix_determinent(temp)/D

    temp = a.copy()
    temp[:,2] = b
    x3 = matrix_determinent(temp)/D

    return x1, x2, x3


In [27]:
A = np.array([[0.3, 0.52, 1], [0.5, 1, 1.9], [0.1, 0.3, 0.5]])
B = np.array([-0.01, 0.67, -0.44])

In [28]:
cramer(A,B)

(-14.900000000000057, -29.500000000000068, 19.80000000000005)

Para mais de três equações, a regra de Cramer torna-se impraticável, pois à medida
que o número de equações cresce, os determinantes demoram mais para ser calculados à
mão (ou pelo computador). Conseqüentemente, são usadas alternativas mais eficientes.

## Eliminação de Gauss Ingênua

Esse método é baseado tecnicas sistematicas para eliminação as variaveis de um sistema progressivamente e substituição regressiva. Apesar de essas técnicas serem hipoteticamente adequadas para implementação em computadores, algumas modificações serãon ecessárias para se obter um algoritmo confiável. Em particular, o programa de computador deve evitar a divisão por zero. O método a seguir é chamado de eliminação de Gauss “ingênua” porque não evita esse problema. Seções subseqüentes irão tratar das características adicionais necessárias para um programa de computador efetivo.
Vamos ver esse método resolvendo o exemplo anterior

$\begin{align*}
3 x_1 - 0.1 x_2 - 0.2 x_3 &= 7.85 \\
0.1 x_1 + 7x_2 - 0.3 x_3 &= -19.3 \\
0.3 x_1 - 0.2 x_2 + 10 x_3 &= 71.4
\end{align*}$

A primeira fase é projetada para reduzir o conjunto de equações a um sistema triangular superior. Para fazer isso vamos eliminar a variavel $x_1$ da segunda equação e $x_1, x_2$ da terceira equação. 

Multiplicamos a primeira linha por $\frac{0.1}{3}$ e depois subtraimos a segunda linha da primeira. O resultado é a nova segunda linha

$\begin{align}
\text{Nova segunda linha}&: (0.1 x_1 + 7x_2 - 0.3 x_3 = -19.3) \\ 
& - \frac{0.1}{3}\times(3 x_1 - 0.1 x_2 - 0.2 x_3 = 7.85) \\
&= 7.00333 x_2 - 0.293333 x_3 = -19.5617
\end{align}$

Depois dessa operação o conjunto de equações é 

$\begin{align}
3 x_1 - 0.1 x_2 - 0.2 x_3 &= 7.85 \\
7.00333 x_2 - 0.293333 x_3 &= -19.5617 \\
0.3 x_1 - 0.2 x_2 + 10 x_3 &= 71.4
\end{align}$

Na proxima etapa eliminamos $x_1$ da terceira equação. Para fazer isso multiplicamos a primeira linha por $\frac{0.3}{3}$ e subtraimos da terceira linha. O resultado fica

$$
\begin{align}
3 x_1 - 0.1 x_2 - 0.2 x_3 &= 7.85 \\
7.00333 x_2 - 0.293333 x_3 &= -19.5617 \\
- 0.1900 x_2 + 10.0200 x_3 &= 70.6150
\end{align}
$$

O ultimo passo da primeira fase é eliminar $x_2$ da terceira equação. Multiplicamos a segunda equação por $\frac{-.1900}{7.0033}$ e subtraimos da terceira. Isso elimina $x_2$ da terceira equação e reduz o sistema a um sistema triangular superior, como em

$\begin{align}
3 x_1 - 0.1 x_2 - 0.2 x_3 &= 7.85 \\
7.00333 x_2 - 0.293333 x_3 &= -19.5617 \\
 10.120 x_3 &= 70.0843
\end{align}$

Agora, é possível resolver essas equações por substituição regressiva. Primeiro, a
terceira equação pode ser resolvida para determinar

$x_3 = \frac{70.843}{10.0120} = 7.000$

Esse resultado pode ser substituído na segunda equação

$7.0033 x_2 - 0.293333(7.000) = -2.5000$

Finalmente os valores de $x_3$ e $x_2$ podem ser inseridos na primeira equação

$3x_1 - 0.1(-2.5000) - 0.2(7.0000) = 7.85 = 3.000$

Tais resultados são idênticos à solução exata. Podemos testar o resultado inserindo os valores no sistema.

Esse metodo é facilmente estendido para uma sistema com $n$ equações e $n$ variaveis. 

Para uma sistema com $n$ variaveis o numero das operações com ponto flutuante (ou flops) para resolver o sistema usando o metodo de eliminação de Gauss é da ordem 

$\underbrace{\frac{2n^3}{3} +\mathcal{O}(n^2)}_{\text{Eliminação progressiva}} +  \underbrace{n^2 +\mathcal{O}(n)}_{\text{Substituição regressiva}} \overbrace{\longrightarrow}^{\text{quando n cresce}} \frac{2n^3}{3} +\mathcal{O}(n^2)$

Agora vamos tentar implementar este algoritmo. O pseducode para este algoritmo é 

<img src="./images/gauss_elimi_simple.jpg" style="width:250px;height:350px;">

In [29]:
def gauss_elimination_minimal(A, b):
    assert A.shape[0] == A.shape[1], 'the matrix of coefficients must be squared'

    mat = np.concatenate([A, b.reshape(-1,1)], axis=1)
    for idx_i in range(0, len(mat)-1):
        for idx_j in range(idx_i+1, len(mat)):
            factor = mat[idx_j, idx_i] / mat[idx_i, idx_i] 
            mat[idx_j] = mat[idx_j] - factor * mat[idx_i]

    n = len(A)-1
    xs = np.zeros(len(A))
    xs[-1] = mat[n,n+1]/mat[n,n]

    for i in range(n-1, -1, -1):
        sum = mat[i, -1]
        for j in range(0, len(A)):
            sum -= mat[i, j]* xs[j]
        xs[i] = sum / mat[i,i]

    return xs

In [30]:
AA = np.array([[3, -.1, -0.2], [0.1, 7, -0.3], [0.3, -0.2, 10]])
bb = np.array([7.85, -19.3, 71.4])

In [31]:
gauss_elimination_minimal(AA, bb)

array([ 3. , -2.5,  7. ])

## Armadilhas dos métodos de eliminação

Embora existam muitos sistemas de equações que podem ser resolvidas pela eliminação
de Gauss ingênua, há algumas armadilhas que precisam ser exploradas antes de escrever
um programa de computador geral para implementar o método.

### Divisão por zero

A principal razão para que a técnica anterior seja chamada de “ingênua” é que tanto durante a fase de eliminação quanto durante a de substituição é possível ocorrer uma divisão por zero. Por exemplo, se for usada a eliminação de Gauss ingênua para resolver

$$
\begin{align}
2 x_2 - 3 x_3 &= 7 \\
4x_1 + 6 x_2 - 7 x_3 &= -3 \\
2 x_1 -  x_2 + 6 x_3 &= 5
\end{align}
$$

a normalização da primeira linha iria envolver a divisão por $a_{11} = 0$. Também podem surgir problemas quando um coeficiente está muito próximo de zero. A técnica de pivotamento foi desenvolvida para evitar parcialmente esses problemas.

### Erros de Arredondamento
Esse erro vem da limitação dos computadores em representar os numeros. Podemos melhorar esse erro usando mais algarismos significativos e usar o padrão de precisão dupla para variaveis.

O problema de erros de arredondamento pode tornar-se particularmente importante
quando se resolve um número grande de equações, por causa do fato de que cada resultado depende dos resultados anteriores. Conseqüentemente, um erro nas etapas iniciais tende a se propagar — isto é, irá causar erros nas etapas subseqüentes. Uma regra empírica é que os erros de arredondamento podem ser importantes quando se lida com 100 ou mais equações.

### Sistemas Mal Condicionados
*Sistemas mal condicionados* são aqueles para os quais pequenas mudanças nos coeficientes resultam em grandes mudanças nas soluções. Uma interpretação alternativa do mal condicionamento é que uma gama ampla de respostas pode satisfazer aproximadamente as equações. Vamos ver isso atraves de um exemplo:

#### **Exemplo:** Resolva o seguinte sistema

\begin{align*}
x_1 + 2x_2 &= 10 \\
1.1 x_1 + 2x_2 &= 10.4
\end{align*}

A seguir, resolva novamente com o coeficiente de $x_1$ na segunda equação levemente modificado para $1.05$.

**Resolução:**

In [32]:
AA = np.array([[1, 2], [1.1, 2]])
bb = np.array([10, 10.4])

In [33]:
gauss_elimination_minimal(AA, bb)

array([4., 3.])

In [34]:
AAA = np.array([[1, 2], [1.05, 2]])
bbb = np.array([10, 10.4])
gauss_elimination_minimal(AAA, bbb)

array([8., 1.])

O problema é que as duas soluções (quase) satisfazem as equações. Assim, embora $x_1 = 8$ e $x_2 = 1$ não sejam a solução verdadeira, a verificação de erro é suficientemente próxima para possivelmente levá-lo a concluir, de maneira errônea, que essa solução é adequada.

Para detectar se um sistema é mal condicionado temos que mudar a escala do sistema e calcular o determinante. Neste caso se o determinante for muito proximo de zero implica que o sistema é mal condicionado. Mas o que é mudar a escala de um sistema? Basicamente é normalizar cada equação dividindo ela por a sua maior coeficiente. 

#### **Exemplo:** Faça uma mudança de escala nos sistemas de equações abaixo de calcula o seu determinante
$$
x_1 + 2x_2 = 10 \\
1.1 x_1 + 2x_2 = 10.4
$$

**Resolução:**

\begin{align*}
\frac{1}{2} (x_1 + 2x_2 &= 10) \\
\frac{1}{2} (1.1 x_1 + 2x_2 &= 10.4)
\end{align*}

\begin{align*}
\rightarrow \frac{x_1}{2} + x_2 &= 5 \\
\rightarrow  \frac{1.1 x_1}{2} + x_2 &= 5.2
\end{align*}


In [35]:
AA = np.array([[0.5, 1], [1.1/2, 1]])
matrix_determinent(AA)

-0.050000000000000044

Então, foi por isso que o sistema é mal condicionado.

### Sistema singulares

Na seção anterior, aprendemos que uma forma pela qual um sistema de equações pode ser
mal condicionado é quando duas ou mais equações são quase iguais. Obviamente, é ainda
pior quando duas equações são idênticas. Em tais casos, perderíamos um grau de liberdade, e estaríamos tratando do caso impossível de n − 1 equações com n incógnitas. Tais
casos podem não ser óbvios para você, particularmente ao lidar com um conjunto grande
de equações. Conseqüentemente, seria ótimo ter alguma forma de detectar a singularidade automaticamente.

## Tecnicas para melhorar o metodo de eliminação de Gauss

As seguintes técnicas podem ser incorporadas ao algoritmo da eliminação de Gauss ingênua para evitar algumas das armadilhas discutidas na seção anterior.

### Uso de mais algarismo signficativos
O remédio mais simples para o mal condicionamento é usar mais algarismos significativos nos cálculos. Se sua aplicação puder ser estendida para manipular palavras de tamanho maior, tal recurso vai reduzir bastante o problema. Entretanto, deve-se pagar um preço na forma de uso elevado de memória e recursos computacionais associado ao uso.
de precisão estendida

### Pivotamento
Como mencionado anteriormente problemas óbvios ocorrem quando um
elemento pivô é zero, pois o passo de normalização leva a uma divisão por zero. Problemas também podem surgir quando o elemento pivô é muito próximo de zero, em vez de exatamente igual a zero, porque, se a ordem de grandeza do elemento pivô é pequena comparada com a dos outros elementos, então podem ocorrer erros de arredondamento. Assim, antes que cada linha seja normalizada, é vantajoso determinar o maior coeficiente disponível na coluna abaixo do elemento pivô. As linhas podem ser trocadas de modo que o maior coeficiente seja o elemento pivô. Isso é chamado de pivotamento parcial. Se procurarmos o maior elemento também nas colunas, além de nas linhas, e então trocarmos, o processo é chamado de pivotamento completo.

#### **Exemplo:** Use eliminação de Gauss para resolver
$$
0.0003 x_1 + 3.000x_2 = 2.0001 \\
1.0000 x_1 + 1.0000 x_2 = 1.0000
$$
Observe que dessa forma o primeiro elemento pivô, $a_{11} = 0.0003$, é muito próximo de
zero. Então, repita os cálculos, mas use pivotamento parcial, trocando a ordem das
equações. As soluções exatas são $x_1 = 1/3$ e $x_2 = 2/3$.

**Resolução:**

Vamos usar a nossa função "minimal" para resolver esse sistema

In [36]:
AA = np.array([[0.0003, 3.0000], [1.000, 1.000]], dtype=np.float16)
bb = np.array([2.0001, 1.000], dtype=np.float16)
gauss_elimination_minimal(AA, bb)

array([-3.25596184,  0.66699219])

Com uma precisão baixa deu um resultado errado! Mas se usarmos a precisão dupla não teremos problema

In [37]:
AA = np.array([[0.0003, 3.0000], [1.000, 1.000]])
bb = np.array([2.0001, 1.000])
gauss_elimination_minimal(AA, bb)

array([0.33333333, 0.66666667])

Mas se a gente usar o pivotamento parcial podemos evitar esse problema, mesmo usando poucos algarismos significaivos.

In [38]:
AA = np.array([[1.000, 1.000], [0.0003, 3.0000]], dtype=np.float16)
bb = np.array([1.000, 2.0001], dtype=np.float16)
gauss_elimination_minimal(AA, bb)

array([0.33349609, 0.66650391])

Um algoritmo para fazer pivotamento parcial é 

<img src="./images/pivo.jpg" style="width:200px;height:350px;">

### Mudando a escala

mudança de escala é valiosa na padronização do
tamanho do determinante. Além dessa aplicação, ela tem utilidade na minimização de
erros de arredondamento para aqueles casos nos quais algumas equações do sistema têm
coeficientes muito maiores que as outras.

## Algoritmo computacional para a eliminação de Gauss

Embaixo tem um algoritmo para implementar o metodo de eliminação de Gauss que já inclui as discussões que a gente tinha sobre as armadilhas do metodo. Observe que o programa inclui módulos para as três operações principais do algoritmo da eliminação de Gauss: eliminação progressiva, substituição regressiva e pivotamento. Além disso, temos diversos aspectos do código que diferem e são aperfeiçoamentos dos pseudocódigos anteriores. São eles:
- Não são feitas mudanças de escala nas equações, mas os valores na nova escala dos
elementos são usados para determinar quando o pivotamento deve ser implementado.
- O termo na diagonal é monitorado durante a fase de pivotamento para se detectar a
ocorrência de valores próximos de zero e identificar sistemas singulares. Se o algoritmo
devolver um valor `er = −1`, uma matriz singular foi detectada e os cálculos devem ser
interrompidos. Um valor pequeno para o parâmetro `tol` é escolhido pelo usuário, para
detectar ocorrências próximas de zero.

<img src="./images/gauss_elimi.jpg" style="width:600px;height:800px;">


### **Exercicio:** Implemente esse algoritmo no Python


### **Exemplo:** 
Suponha que um time de três páraquedistas está ligado por uma corda sem peso enquanto cai, em queda livre, a uma velocidade de $5 m/s$. Calcule a tensão em cada seção da corda e a aceleração do time, dado o seguinte:

|Paraquedista | Massa (kh) | Coeficiente de arrasto|
|   ----      | -----------|-----------------------|
| 1           |     70     |        10             |
| 2           |    60      |       14              |
|       3     |     40     |        17             |


<img src="./images/paraq1.jpg" style="width:100px;height:500px;">

<img src="./images/paraq2.jpg" style="width:400px;height:360px;">


Essas equações possuem três incógnitas: $a, T$ e $R$. Depois de substituir os valores conhecidos, as equações podem ser expressas na forma matricial ($g = 9,8 m/s^2$ ),

$$
D = \begin{bmatrix}
70 & 1 & 0\\
60 & -1 & 1 \\
40 & 0 & -1 
\end{bmatrix}

\begin{bmatrix}
a &\\
T & \\
R & 
\end{bmatrix}
=
\begin{bmatrix}
636 &\\
518 & \\
307 & 
\end{bmatrix}
$$ 
Esse sistema pode ser resolvido usando sua própria implementação. O resultado é $a = 8,5941 m/s^2$; $T = 34,4118 N$ e $R = 36,7647 N$.

## Determinante de uma matriz $n \times n$

No metodo de eliminação de Gauss, depois do processo de eliminação progressiva, o determinante pode ser calculada simplesmente por 

$$
D = a_{11} a'_{22} a''_{33} ... a^{n-1}_{nn}
$$
onde os subscritos significam o número de vezes que os elementos foram modificados pelo processo de eliminação. Assim, podemos aproveitar o esforço que já foi despendido para reduzir o sistema a uma forma triangular e, de brinde, obter uma estimativa simples do
determinante.

### **Exercicio:** Implemente esse algoritmo de calcular determinante no Python

## Gauss-Jordan

O método de Gauss-Jordan é uma variação da eliminação de Gauss. A maior diferença é
que, quando uma variável é eliminada no método de Gauss-Jordan, ela é eliminada de
todas as outras equações, não só das posteriores. Além disso, todas as linhas são normalizadas pela divisão pelo seu elemento pivô. Então, o passo de eliminação resulta na matriz identidade e não em uma matriz triangular. Vamos ver esse metodo atraves de um exemplo

### **Exemplo:** 
Usando o metodo Gauss-Jordan resolve o sistema

$\begin{align*}
3 x_1 - 0.1 x_2 - 0.2 x_3 &= 7.85 \\
0.1 x_1 + 7x_2 - 0.3 x_3 &= -19.3 \\
0.3 x_1 - 0.2 x_2 + 10 x_3 &= 71.4
\end{align*}$ 

**Resolução:**

Primeiro, expresse os coeficientes e as constantes do lado direito da equação
como uma matriz aumentada:

$$
\begin{bmatrix}
3 & -0.1 & -0.2 & 7.85\\
0.1 & 7 & -0.3 & -19.3\\
0.3 & -0.2 & 10 & 71.4 
\end{bmatrix}
$$

Então, normalize a primeira linha dividindo-a pelo elemento pivô, 3, para obter

$$
\begin{bmatrix}
1 & -0.033333 & -0.066667 & 2.61667\\
0.1 & 7 & -0.3 & -19.3\\
0.3 & -0.2 & 10 & 71.4 
\end{bmatrix}
$$

O termo $x_1$ pode ser eliminado da segunda linha subtraindo 0,1 vezes a primeira linha da segunda linha. De maneira semelhante, subtraindo 0,3 vez a primeira linha da terceira linha,
iremos eliminar o termo $x_1$ da terceira linha:

$$
\begin{bmatrix}
1 & -0.033333 & -0.066667 & 2.61667\\
0 & 7.00333 & -0.293333 & -19.5617\\
0 & -0.190000 & 10.0200 & 70.6150 
\end{bmatrix}
$$

Em seguida, normalizamos a segunda linha, dividindo-a por 7,00333:

$$
\begin{bmatrix}
1 & -0.033333 & -0.066667 & 2.61667\\
0 & 1 & -0.0418848 & -2.79320\\
0 & -0.190000 & 10.0200 & 70.6150 
\end{bmatrix}
$$

A redução do termo $x_2$ da primeira e da terceira linhas resulta em

$$
\begin{bmatrix}
1 & 0 & -0.0680629 & 2.52356\\
0 & 1 & -0.0418848 & -2.79320\\
0 & 0 & 10.01200 & 70.0843 
\end{bmatrix}
$$

A terceira linha é então normalizada dividindo-se por 10,0120:

$$
\begin{bmatrix}
1 & 0 & 0 & 3.000\\
0 & 1 & 0 & -2.5000\\
0 & 0 & 1 & 7.0000 
\end{bmatrix}
$$

Assim, a matriz dos coeficientes foi transformada na matriz identidade e a solução é obtida no vetor do lado direito. Observe que não foi necessária a substituição regressiva para se obter a solução.

Toda a discussão relativo a armadilhas e aprimoramentos do método de eliminação de Gauss também se aplica ao método de Gauss-Jordan. Por exemplo, uma estratégia de pivotamento parecida pode ser usada para evitar a divisão por zero e reduzir erros de arredondamento.

Computacionalmente o metodo de Gauss-Jordan é 50% mais pesado do que o metodo de eliminação de Gauss

$\frac{n^3}{2} + n^2 - \frac{n}{2} \overbrace{\longrightarrow}^{\text{quando n cresce}} \frac{n^3}{2} +\mathcal{O}(n^2)$


## Exercicio: Implemente of metodo Gauss-Jordan no Python

## Decomposição LU

Este metodo é muito parecido com a eliminação de Gauss. Tem muitas formas de decomposição de uma matriz mas aqui a gente vamos estudar a decomposição LU. De fato, o metodo de eleiminação pode ser implementado como uma decomposição LU.

O interesse principal da decomposição LU é que o passo de eliminação,
que consome muito tempo, pode ser formulado de modo que envolva apenas operações
na matriz dos coeficientes, [A]. Assim, ele se adapta bem àquelas situações nas quais
muitos vetores à direita, {b}, devem ser calculados para um único valor de [A].

Outro motivo para aprender a decomposição LU é que ela fornece uma maneira eficiente de calcular a matriz inversa, a qual tem um grande número de aplicações valiosas
na prática da engenharia. A decomposição LU também fornece um meio de avaliar o
condicionamento do sistema.

Na composição LU, no primeiro passo a gente vai dividir a matriz [A] em duas matrizes triangulares, [L] e [U]. A matriz U é a matriz que calculamos no final do processo de eliminação. Agora a matriz [L] é uma matrix diagonal e os elementos não-zeros são os fatores que usamos no processo de eliminação para criar a matriz triangular [U]. Então o primeiro passo é decompor a matriz em duas matrizes triangulares. 
$$
A = L U
$$
A matriz [U] satisfaz a seguinte equação
$$
U X - d = 0
$$
onde ${d}$ é um vetor que temos que calcular. E a matriz [L] satisfaz
$$
L d = b
$$
Juntando essas duas condições conseguimos chegar na equação inicial 
$$
L(U X - d) = 0 \rightarrow (L U) X- L d = A X - b = 0
$$
O segundo passo é usar a relação $L d = b$ para determinar o vetor intermediario ${d}$ e depois usar esse vetor na relação $U X -d = 0$. 

Resumindo as duas etapas temos:

- Passo da decomposição LU. [A] é fatorada ou “decomposta” em matrizes triangu-
lares inferior [L] e superior [U].
- Passo de substituição. [L] e [U] são usadas para determinar a solução {X} para um
lado direito {b}. Esse passo consiste em duas etapas. Primeiro, a $L d = b$ é
usada para gerar um vetor intermediário {d} por substituição progressiva. A seguir,
o resultado é substituído na $U X -d = 0$, que pode ser resolvida por substituição
regressiva, determinando-se {X}.

Graficamente esse processo pode ser demonstrado na seguinte forma

<img src="./images/lu_graph.jpg" style="width:600px;height:400px;">

## A Versão da Eliminação de Gauss por Decomposição LU
Na eliminção de Gauss, depois de terminar o processo de eliminação, para uma matriz $3 \times 3$, temos algo do tipo

$$
[U] = \begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
0 & a'_{22} & a'_{23}\\
0 & 0 & a''_{33} 
\end{bmatrix}
$$
onde $a'$ e $a''$ são elementos que foram calculados na primeira e segunda fase de eliminação. Agora como calcular [L]? Considerem um sistema $3 \times 3$ geral

$$
\begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23}\\
a_{31} & a_{32} & a_{33} 
\end{bmatrix}

\begin{bmatrix}
x_{1} & \\
x_{2} & \\
x_{3}  
\end{bmatrix}

=

\begin{bmatrix}
b_{1} & \\
b_{2} & \\
b_{3}  
\end{bmatrix}
$$
A primeira etapa da eliminação de Gauss é multiplicar a linha 1 pelo fator
$$
f_{21} = \frac{a_{21}}{a_{11}}
$$
e subtrair o resultado da segunda linha para eliminar $a_{21}$ . Analogamente, a linha 1 é mul-
tiplicada por
$$
f_{31} = \frac{a_{31}}{a_{11}}
$$
e o resultado é subtraído da terceira linha para eliminar $a_{31}$ . O passo final é multiplicar a
segunda linha modificada por
$$
f_{31} = \frac{a'_{32}}{a'_{22}}
$$
e subtrair o resultado da terceira linha para eliminar $a'_{32}$. Aí, a matriz [L] é 

$$
[L] = \begin{bmatrix}
1 & 0 & 0\\
f_{21} & 1 & 0\\
f_{31} & f_{32} &  1
\end{bmatrix}
$$

Vamos ver esse procedimento em um exemplo:

### **Exemplo:** Obtenha uma decomposição LU baseada na eliminação de Gauss efetuada para a matriz

$$
\begin{bmatrix}
3 & -0.1 & -0.2\\
0.1 & 7 & -0.3\\
0.3 & -0.2 & 10 
\end{bmatrix}
$$

**Resolução:** 

A matriz [U] já foi calculada e é 
 
$$
U = \begin{bmatrix}
3 & -0.1 & -0.2\\
0 & 7.0033 & -0.2933\\
0 & 0 & 10.0120 
\end{bmatrix}
$$ 

Os fatores empregados para obter a matriz triangular superior podem ser reunidos
em uma matriz triangular inferior. Os elementos $a_{21}$ e $a_{31}$ foram eliminados usando-se
os fatores

$$
f_{21} = \frac{0.1}{3} = 0.03333 \;\;f_{31} = \frac{0.3}{3} = 0.10000
$$
e o elemento $a'_{32}$ foi eliminado usando-se o fator
$$
f_{32} = \frac{-0.19}{7.0033} = -0.0271300
$$

Assim, a matriz [L] é 

$$
L = \begin{bmatrix}
1 & 0 & 0\\
0.03333 & 1 & 0\\
0.10000 & -0.0271300 & 1 
\end{bmatrix}
$$ 

Vamos implementar isso no Python

In [3]:
def decompose(A):
    assert A.shape[0] == A.shape[1], 'the matrix of coefficients must be squared'

    L = np.eye(len(A))
    U = A.copy()

    n = len(A)
    for idx_i in range(0, n-1):
        for idx_j in range(idx_i+1, n):
            factor = U[idx_j, idx_i] / U[idx_i, idx_i] 
            U[idx_j] = U[idx_j] - factor * U[idx_i]
            L[idx_j, idx_i] = factor
    

    return L, U

    

In [4]:
AA = np.array([[3, -.1, -0.2], [0.1, 7, -0.3], [0.3, -0.2, 10]])
decompose(AA)

(array([[ 1.        ,  0.        ,  0.        ],
        [ 0.03333333,  1.        ,  0.        ],
        [ 0.1       , -0.02712994,  1.        ]]),
 array([[ 3.        , -0.1       , -0.2       ],
        [ 0.        ,  7.00333333, -0.29333333],
        [ 0.        ,  0.        , 10.01204188]]))

Depois que a matriz é decomposta, pode-se produzir uma solução para um valor particular do vetor à direita {b}, o que é feito em duas etapas. Primeiro, uma substituição
progressiva é executada, resolvendo-se a $L d = b$ para {d}. É importante reconhecer que isso significa simplesmente efetuar as manipulações da eliminação em {b}.
Assim, ao final desse passo, o lado direito estará no mesmo estado em que estaria se
tivesse sido efetuada a eliminação progressiva em [A] e {b} simultaneamente.
O passo de substituição progressiva pode ser representado de modo conciso por

$$
d_i = b_i - \sum^{i-1}_{j=1} a_{ij}d_j \;\;\; \text{para } i=2,3,...,n
$$

O segundo passo, então, resume-se simplesmente em implementar a substituição regressiva, como na $U X - d = 0$. Novamente, é importante perceber que isso é idêntico à fase de substituição regressiva da eliminação de Gauss convencional.

$$
x_n = \frac{d_n}{a_{nn}} \\
x_i = \frac{d_i - \sum^{n}_{j=i+1} a_{ij}x_j}{a_{ii}} \;\;\; \text{para } i=n-1,n-2,...,1
$$

### **Continuação do Exemplo:** Complete o problema iniciado, determinando a solução final por substituição progressiva e regressiva.

**Resolução:**

Primeiro vamos resolver a equação $L d = b$, ou seja:

$$
\begin{bmatrix}
1 & 0 & 0\\
0.03333 & 1 & 0\\
0.10000 & -0.0271300 & 1 
\end{bmatrix}

\begin{bmatrix}
d_1 \\
d_2 \\
d_3 
\end{bmatrix}
=
\begin{bmatrix}
7.85 \\
-19.3 \\
71.4 
\end{bmatrix}
$$ 

Multiplicando o lado esquerdo temos

$$
d_1 = 7.85 \\
0.03333d_1 + d_2 = -19.3 \\
0.1 d_1 - 0.02713 d_2 + d_3 = 71.4
$$

Da primeira equação temo $d_1 = 7.85$. Substituindo isso na segunda equação temos

$$
d_2 = -19.3 - 0.0333(7.85) = -19.5617
$$

E finalmente inserindo $d_1, d_2$ na terceira equação achamos o valor da $d_3$

$$
d_3 = 71,4 − 0,1(7,85) + 0,02713(−19,5617) = 70,0843
$$

Assim,

$$
\begin{bmatrix}
7.85 \\
-19.5617 \\
70.0843 
\end{bmatrix}
$$ 

Agora, a ultima fase para calcular os valores de $x$ é identico ao processo de substituição regressiva na eliminação de Gauss
$$
\begin{bmatrix}
3 & -0.1 & -0.2\\
0 & 7.0033 & -0.2933\\
0 & 0 & 10.0120 
\end{bmatrix}

\begin{bmatrix}
x_1 \\
x_2 \\
x_3 
\end{bmatrix}

=

\begin{bmatrix}
7.85 \\
-19.5617 \\
70.0843 
\end{bmatrix}
$$

Esse procedimento já foi feito anteriormente e não vamos reproduzir aqui.

Agora vamos implementar essa fase no Python

In [19]:
def substitute(L, U, b):
    # forward substitution
    n = len(U)
    ds = np.zeros(n)

    for i in range(0, n):
        ds[i] = b[i] - sum(L[i] * ds)
    
    xs = np.zeros(n)
    for i in range(n-1, -1, -1):
        xs[i] = (ds[i] - sum(U[i] * xs)) / U[i, i]

    return xs

In [20]:
AA = np.array([[3, -.1, -0.2], [0.1, 7, -0.3], [0.3, -0.2, 10]])
bb = np.array([7.85, -19.3, 71.4])
L, U = decompose(AA)
substitute(L, U, bb)

array([ 3. , -2.5,  7. ])

O algoritmo de decomposição LU exige o mesmo número de flops de multiplicação/divisão que a eliminação de Gauss. A única diferença é que um pouco menos de esforço é gasto na fase de decomposição, já que as operações não são aplicadas ao lado direito.

## A Matriz Inversa

A matriz inversa é uma matriz que satisfaz a seguinte relação

$$
A A^{-1} = A^{-1} A = I
$$

Onde $I$ é uma matriz diagonal com elementos 1. Po]r exemplo para um caso de uma matriz $3 \times 3$ temos

$$
I = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1 
\end{bmatrix}
$$

Podemos usar o metodo de composição LU para calcular a matriz inversa. Para fazer isso, separamos as colunas da matriz $I$ e cada coluna será um vetor {b}. Alem disso, vamos fazer a decomposição da matriz [A]. Por exemplo, para achar a primeira coluna da matriz inversa para uma matriz $3 \times 3$ vamos ter as seguintes relações
$$

[A] = [L][U]
$$
$$
{b} = \begin{bmatrix}
1 \\
0 \\
0 
\end{bmatrix} 
$$

$$
[L] \{d\} = \{b\}
$$
$$
[U] \{x\} = \{d\} 
$$

Onde $\{x\}$ é a primeira coluna da matriz inversa da matriz [A] e ${b}$ é a primeira coluna da matriz $[I]$. Vamos ver esse procedimento via um exemplo

### **Exemplo:** Utilize a decomposição LU para determinar a inversa da matriz

$$
\begin{bmatrix}
3 & -0.1 & -0.2\\
0.1 & 7 & -0.3\\
0.3 & -0.2 & 10 
\end{bmatrix}
$$

Lembre-se de que a decomposição resultou nas seguintes matrizes triangulares superior e
inferior:

$$
L = \begin{bmatrix}
1 & 0 & 0\\
0.03333 & 1 & 0\\
0.10000 & -0.0271300 & 1 
\end{bmatrix}
$$ 

$$
U = \begin{bmatrix}
3 & -0.1 & -0.2\\
0 & 7.0033 & -0.2933\\
0 & 0 & 10.0120 
\end{bmatrix}
$$ 

A primeira coluna da matriz inversa é determinada 

$$
\begin{bmatrix}
1 & 0 & 0\\
0.03333 & 1 & 0\\
0.10000 & -0.0271300 & 1 
\end{bmatrix}
\begin{bmatrix}
d_1 \\
d_2 \\
d_3 
\end{bmatrix}
 = 
\begin{bmatrix}
1 \\
0 \\
0 
\end{bmatrix}
$$ 

e resolvido com substituição progressiva para determinar $\{D\}^T = [1, −0.03333,
−0.1009]$. Esse vetor pode ser usado como lado direito na equação $[U]\{x\} = \{d\}$

$$
\begin{bmatrix}
3 & -0.1 & -0.2\\
0 & 7.0033 & -0.2933\\
0 & 0 & 10.0120 
\end{bmatrix}

\begin{bmatrix}
x_1 \\
x_2 \\
x_3 
\end{bmatrix}
 = 
\begin{bmatrix}
1 \\
-0.0333 \\
-0.1009
\end{bmatrix}
$$ 

a qual pode ser resolvida por substituição regressiva para determinar $\{X\}^T = [0.33249, −0.00518, −0.01008]$, que é a primeira coluna da matriz,

$$
A^{-1} = \begin{bmatrix}
30.33249& ? & ?\\
-0.00518 & ? & ?\\
-0.01008 & ? & ? 
\end{bmatrix}
$$

Continuando esse procedimento para outras colunas chegamos na matriz inversa completa

$$
A^{-1} = \begin{bmatrix}
0.33249 & 0.004944 & 0.006798\\
-0.00518 & 0.142903 & 0.004183\\
-0.01008 & 0.00271 & 0.09988 
\end{bmatrix}
$$

## **Exercicio:** Implemente o algoritmo da matriz inversa usando decomposição LU no Python



## Gauss-Seidel

Os métodos iterativos ou de aproximação fornecem uma alternativa aos métodos de
eliminação descritos até agora. Tais abordagens são semelhantes às técnicas desenvolvidas para obter as raízes de uma única equação. Aquelas abordagens consistiam em escolher um valor e então usar um método sistemático para obter uma estimativa refinada da raiz. O procedimento é o seguinte. Suponha que tenhamos um conjunto de $n$ equações

$$
[A]\{x\} = \{b\}
$$

No caso de um matriz $3 \times 3$, se os elementos da diagonal forem todos não-nulos, esse sistema pode ser escrito na forma

$$
\begin{align*}
x_1 &= \frac{b_1 - a_{12}x_2 - a_{13}x_3}{a_{11}} \\
x_2 &= \frac{b_2 - a_{21}x_1 - a_{23}x_3}{a_{22}} \\
x_3 &= \frac{b_3 - a_{31}x_1 - a_{32}x_2}{a_{33}} 
\end{align*}
$$

Agora, a partir de um chute incial podemos resolver essas equações um por um. Começando com a euqação para $x_1$, temos
$$
x_1^{i+1} = \frac{b_1 - a_{12}x_2^i - a_{13}x_3^i}{a_{11}} 
$$

calculando essa equação, inserimos esse valor novo de $x_1$ na segunda equação

$$
x_2^{i+1} = \frac{b_2 - a_{21}x_1^{i+1} - a_{23}x_3^{i}}{a_{22}} 
$$

e finalmente para a variavel $x_3$ temos

$$
x_3^{i+1} = \frac{b_3 - a_{31}x_1^{i+1} - a_{32}x_2^{i+1}}{a_{33}} 
$$

Então, volta-se para a primeira equação e o procedimento inteiro é repetido até que a
solução convirja para valores suficientemente próximos dos valores verdadeiros. A convergência pode ser verificada usando-se o critério

$$
|\varepsilon_{a, i}| = |\frac{x_i^j - x_i^{j-1}}{x_i^j}| 100\% < \varepsilon_s
$$
para todo $i$, onde $j$ e $j − 1$ representam a iteração atual e a anterior.

Vamos implementar esse algoritmo no Python

In [70]:
def gauss_seidel(A, b, x0, tol=1e-5, max_iter=20):
    n = len(A)
    assert len(x0) == n, 'the size of initial guess must be the same as the system'
    assert all([A[i,i] != 0 for i in range(n)]), 'There is a zero in the diagonal of the matrix'
    iteration = 0
    x_old = xs = np.array(x0, dtype=np.float64)
    error = np.ones(n)*100

    while max(np.abs(error)) > tol and iteration < max_iter:
        
        for i in range(n):
            xs[i] = (b[i] - sum(np.delete(xs, i) * np.delete(A[i, :], i)))/A[i, i]
            error[i] = np.abs((xs[i] - x_old[i])/ xs[i])*100
            x_old = xs.copy()
        
        iteration += 1

    return xs


In [71]:
AA = np.array([[3, -.1, -0.2], [0.1, 7, -0.3], [0.3, -0.2, 10]])
bb = np.array([7.85, -19.3, 71.4])
x0 = [10,5,100]

gauss_seidel(AA, bb, x0, tol=1e-10, max_iter=10)

array([ 3. , -2.5,  7. ])

## Metodo de iteração de Jacobi

À medida que cada novo valor de x é calculado pelo método de Gauss-Seidel, ele é
imediatamente usado na próxima equação para se determinar outro valor de $x$. Assim, se
a solução estiver convergindo, a melhor estimativa disponível será empregada. Uma
abordagem alternativa, chamada iteração de Jacobi, utiliza uma tática um pouco diferente. Em vez de usar os últimos $x$'s disponíveis, essa técnica calcula um conjunto de novos $x$'s com base no conjunto de antigos $x$'s. Logo, conforme novos valores são gerados, não são imediatamente usados, mas guardados para a próxima iteração.

<img src="./images/gauss_jacobi.jpg" style="width:450px;height:400px;">

## Exercicio: Implemente o metodo de Jacobi no Python

### Convergencia do metodo de Gauss-Seidel (e de Jacobi)

#### **Critero das linha (metodo de Jacobi e Gauss-Seidel)**
Seja o sistema linear $[A]\{x\} = \{b\}$ e seja $\alpha_k = \sum_{\substack{i=1 \\ i\neq k}}^n |a_{ki}|/|a_{kk}|$. Se $\alpha = \text{max}_{1 \le k \le n} \alpha_k <1 $, então o metodo de Gauss-Seidel gera uma sequencia $\{x^k\}$ convergente para a solução do sistema dado, independentemente da scolha da aproximação inicial.

#### **Critero de Sassenfeld (metodo de Gauss-Seidel)**
Seja o sistema linear $[A]\{x\} = \{b\}$. Sejam 

$$
\beta_1 = \frac{|a_{12}| + |a_{13}| + ... +|a_{1n}|}{|a_{11}|}\\

\beta_j = \frac{|a_{j1}|\beta_1 + |a_{j2}|\beta_2 + ...+ |a_{jj-1}|\beta_{j-1} +|a_{jj+1}|+...+|a_{jn}|}{|a_{11}|}
$$
Seja $\beta = \text{max}_{1 \le j \le n}\{\beta_j\}$.

Se $\beta < 1$, então o metodo de Gauss-Seidel gera uma sequencia convergente qualquer que seja a aproximação inicial. Alem disso, quanto menor for $\beta$, mais rapida será a convergência. 